In [ ]:
import pandas as pd
df = pd.read_csv(r'C:\Users\mtsil\OneDrive\Documents\finalcorpus.csv', ';', encoding= 'unicode_escape') #call newdf as df1
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
def clean_text(text):
    text = text.lower()  
    
    text = " ".join(text.split())
    return text


df['clean']= df['text'].map(clean_text)

df

In [ ]:
df['text_clean'] = df['clean'].str.replace(r'[^\w\s]+', '')
df

In [ ]:
import contractions
df['cleaned'] = df['text_clean'].apply(lambda x: contractions.fix(x))
df

In [ ]:
df.to_csv(r'C:\Users\mtsil\OneDrive\Documents\cleaneddoc.csv', ';') 

In [ ]:
df1 = pd.read_csv(r'C:\Users\mtsil\OneDrive\Documents\cleaneddoc.csv', ';', encoding= 'unicode_escape') #call newdf as df1
df1.head()

In [ ]:
df1['label'].value_counts()

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
train, test = train_test_split (df1, test_size=.20, random_state=42)

tagged_tr = [TaggedDocument(words=word_tokenize(_d.lower()),
tags=[str(i)]) for i, _d in enumerate(train.text)]

tagged_test = [TaggedDocument(words=word_tokenize(_d.lower()),
tags=[str(i)]) for i, _d in enumerate(test.text)]

In [ ]:
model = Doc2Vec(vector_size=300,
                window=5, 
                alpha=.025, 
                min_alpha=0.00025, 
                min_count=2, 
                dm=0, 
                workers=8,
               negative = 10)
model.build_vocab(tagged_tr)

In [ ]:
epochs = range(20)
for epoch in epochs:
    print(f'Epoch {epoch+1}')
    model.train(tagged_tr,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    
    model.alpha -= 0.00025
    
    model.min_alpha = model.alpha

In [ ]:
import numpy as np
X_train = np.array([model.dv[str(i)] for i in range(len(tagged_tr))])
y_train = train['label']

In [ ]:
X_test = np.array([model.infer_vector(tagged_test[i][0]) for i in range(len(tagged_test))])
y_test = test['label']

In [ ]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm  = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(classification_report(y_test,y_pred))